In [ ]:
%pip install tensorflow
%pip install numpy
%pip install pandas
%pip install ast
%pip install logging
%pip install ast
%pip install seaborn
%pip install mlflow
%pip install imbalanced-learn

In [ ]:
import logging
import numpy as np
import pandas as pd
import ast
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
# Initialize logging
logging.basicConfig(level=logging.INFO)


def preprocess_data(data_path):
    """
    Preprocesses the data by loading, validating, and filtering it.
    Args: data_path (str): Path to the CSV file containing data.
    Returns: tuple: (padded_sequences, labels, classes)
    """
    logging.info('Starting data preprocessing')

    try:
        # Load the data
        df = pd.read_csv(data_path)

        # Validate required columns
        required_columns = ['Babbles', 'Sex']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing columns: {missing_columns}")

        # Convert strings to lists and filter by length
        df['Babbles'] = df['Babbles'].apply(ast.literal_eval)
        df = df[df['Babbles'].apply(lambda x: 50 <= len(x))]

        # Pad sequences
        sequences = df['Babbles'].values
        padded_sequences = tf.keras.utils.pad_sequences(sequences, padding='post', dtype='float32')

        # Encode labels
        le = LabelEncoder()
        labels = le.fit_transform(df['Sex'])

        logging.info(f'Processed {len(padded_sequences)} valid sequences with a length > 50')
        return padded_sequences, labels, le.classes_

    except FileNotFoundError:
        logging.error(f"File {data_path} not found.")
        raise
    except Exception as e:
        logging.error(f"Error during preprocessing: {e}")
        raise


def focal_loss(gamma=2., alpha=0.25):
    """Focal loss for addressing class imbalance."""
    def focal_loss_fixed(y_true, y_pred):
        y_pred = tf.keras.backend.clip(y_pred, 1e-9, 1 - 1e-9)
        loss = -alpha * (1 - y_pred) ** gamma * y_true * tf.keras.backend.log(y_pred)
        return tf.keras.backend.mean(tf.keras.backend.sum(loss, axis=1))
    return focal_loss_fixed


from tensorflow.keras.layers import Attention

def create_model(input_length, num_classes):
    input_seq = tf.keras.layers.Input(shape=(input_length, 1))

    # First BiLSTM layer
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(input_seq)

    # Add attention layer
    x = Attention()([x, x]) 

    # Second BiLSTM layer   
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    # Dense Layer with L2 Regularization    
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    # x = tf.keras.layers.Dropout(0.3)(x)

    # Output Layer
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=input_seq, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=focal_loss(),
        metrics=['accuracy']
    )
    return model



def plot_confusion_matrix(y_true, y_pred, classes):
    """Plots the confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()


class TrainingCallback(tf.keras.callbacks.Callback):
    """Custom callback for detailed training progress."""
    def on_epoch_end(self, epoch, logs=None):
        print(
            f'Epoch {epoch + 1}: loss = {logs["loss"]:.4f}, '
            f'accuracy = {logs["accuracy"]:.4f}, '
            f'val_loss = {logs["val_loss"]:.4f}, '
            f'val_accuracy = {logs["val_accuracy"]:.4f}')


def handle_class_imbalance(X_train, y_train):
    """Handle class imbalance by oversampling or weighting."""
    y_train_labels = np.argmax(y_train, axis=1)
    class_counts = np.bincount(y_train_labels)
    logging.info(f"Initial class distribution in training data: {dict(zip(np.unique(y_train_labels), class_counts))}")

    if np.any(class_counts < 0.1 * len(y_train_labels)):
        logging.info("Class imbalance detected. Applying SMOTE oversampling.")
        smote = SMOTE(sampling_strategy='auto', random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train_labels)
        y_train_resampled = tf.keras.utils.to_categorical(y_train_resampled, num_classes=y_train.shape[1])
        new_class_counts = np.bincount(np.argmax(y_train_resampled, axis=1))
        logging.info(f"New class distribution after SMOTE oversampling: {dict(zip(np.unique(y_train_resampled), new_class_counts))}")
        return X_train_resampled, y_train_resampled, None
    else:
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train_labels), y=y_train_labels)
        class_weights_dict = {i: weight for i, weight in zip(np.unique(y_train_labels), class_weights)}
        logging.info(f"Class weights: {class_weights_dict}")
        return X_train, y_train, class_weights_dict


def main():
    # Preprocess data
    try:
        X, y, classes = preprocess_data('../CMBabble_Master_combined_scm.csv')
    except Exception as e:
        logging.error(f"Preprocessing failed: {e}")
        return

    # Reshape and split data
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = tf.keras.utils.to_categorical(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Handle class imbalance
    X_train, y_train, class_weights = handle_class_imbalance(X_train, y_train)

    # Create and train model
    model = create_model(X_train.shape[1], len(classes))
    
    callbacks = [
        TrainingCallback(),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.3,
            patience=5, 
            min_lr=5e-7
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath='best_model.keras', 
            monitor='val_loss', 
            save_best_only=True, 
            verbose=1
        ),
        tf.keras.callbacks.TensorBoard(
            log_dir='./logs',
            histogram_freq=1
        )
    ]

    history = model.fit(
        X_train, y_train,
        epochs=50, 
        batch_size=32,
        validation_split=0.2,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=1,
        class_weight=class_weights
    )

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"\nTest Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}\n")

    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)

    plot_confusion_matrix(true_classes, predicted_classes, classes)
    print(classification_report(true_classes, predicted_classes, target_names=classes, zero_division=1))

    return model, history


if __name__ == "__main__":
    main()
